# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,50.74,74,70,1.01,AU,1722813057
1,1,albany,42.6001,-73.9662,81.45,74,44,1.01,US,1722813057
2,2,la tuque,47.4334,-72.7824,67.89,77,93,3.56,CA,1722813057
3,3,hastings,50.8552,0.5729,61.09,86,72,10.18,GB,1722813057
4,4,vilyuysk,63.7553,121.6247,63.03,50,96,7.65,RU,1722813057


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.6,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp'] > 40) & (city_data_df['Max Temp'] < 100)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,50.74,74,70,1.01,AU,1722813057
1,1,albany,42.6001,-73.9662,81.45,74,44,1.01,US,1722813057
2,2,la tuque,47.4334,-72.7824,67.89,77,93,3.56,CA,1722813057
3,3,hastings,50.8552,0.5729,61.09,86,72,10.18,GB,1722813057
4,4,vilyuysk,63.7553,121.6247,63.03,50,96,7.65,RU,1722813057


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,74,
1,albany,US,42.6001,-73.9662,74,
2,la tuque,CA,47.4334,-72.7824,77,
3,hastings,GB,50.8552,0.5729,86,
4,vilyuysk,RU,63.7553,121.6247,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}
#x = 0
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    #x = x + 1
    #if x == 10:
        #break
# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
albany - nearest hotel: No hotel found
la tuque - nearest hotel: Auberge La Résidence
hastings - nearest hotel: White Rock Hotel
vilyuysk - nearest hotel: Мария
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
waitangi - nearest hotel: Hotel Chathams
bredasdorp - nearest hotel: Victoria Hotel
jamestown - nearest hotel: DoubleTree Jamestown
taroa - nearest hotel: No hotel found
chimore - nearest hotel: Aquí me quedo
adamstown - nearest hotel: No hotel found
xilin hot - nearest hotel: Hotel hailibu 120
kievka - nearest hotel: No hotel found
giyani - nearest hotel: No hotel found
kitimat - nearest hotel: MStar Hotel
longyearbyen - nearest hotel: The Vault
udachny - nearest hotel: Вилюй
mizhhirya - nearest hotel: Кооператор
edinburgh of the seven seas - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
hwange - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
iskateley -

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.6,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot1